<a href="https://colab.research.google.com/github/LandonAmickUNCC/ECGR4105/blob/main/Homework7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Landon* Amick 801293791 Homework 7 https://github.com/LandonAmickUNCC/ECGR4105

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from mpl_toolkits import mplot3d
from ipywidgets import interact, fixed
import seaborn as sns; sns.set()
from sklearn.metrics import classification_report
import torch
import torch.nn as nn
import time
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
import torch.optim as optim

**Part 1A: CNN NON**

In [3]:
from torchvision import datasets

import torchvision.transforms as transforms

data_path = './data-unversioned/p1ch7/'

cifar10_val = datasets.CIFAR10(data_path, train=False, download=True, transform=transforms.ToTensor())

transformed_cifar10_train = datasets.CIFAR10(data_path, train=True, download=True, transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.4915,0.4823,0.4468),(0.2470,0.2435,0.2616))]))

100%|██████████| 170M/170M [00:13<00:00, 12.7MB/s]


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [ ]:
def softmax(x):
  return torch.exp(x)/torch.exp(x).sum()

n_out = 10

cifar_model1 = nn.Sequential(nn.Linear(3072,512),nn.ReLU(),nn.Linear(512,n_out),nn.LogSoftmax(dim=1)).to(device)

In [ ]:
learning_rate = 1e-2

optimizer = optim.SGD(cifar_model1.parameters(), lr=learning_rate)

loss_fn_cifar= nn.NLLLoss()

n_epochs = 300

def train_cifar_model1(n_epochs, optimizer, model_nn, train_dataset, val_dataset):
    start_time = time.time()

    # Create DataLoaders for batching
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=64, shuffle=False)

    for epoch in range(1, n_epochs + 1):
        model_nn.train() # Set model to training mode
        running_loss = 0.0
        for imgs, labels in train_loader:
            # Flatten images for the linear model
            imgs = imgs.view(imgs.shape[0], -1).to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            out = model_nn(imgs)
            loss = loss_fn_cifar(out, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        avg_train_loss = running_loss / len(train_loader)

        # Validation phase
        model_nn.eval() # Set model to evaluation mode
        val_running_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for imgs, labels in val_loader:
                imgs = imgs.view(imgs.shape[0], -1).to(device)
                labels = labels.to(device)
                out = model_nn(imgs)
                val_loss = loss_fn_cifar(out, labels)
                val_running_loss += val_loss.item()

                _, predicted = torch.max(out.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        avg_val_loss = val_running_loss / len(val_loader)
        val_accuracy = 100 * correct / total

    end_time = time.time()
    total_training_time = end_time - start_time
    if epoch % 20 == 0:
      print(f'Epoch {epoch}, Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%')
      print(f"Epoch {epoch}, Training Loss: {avg_train_loss:.4f}")
      print(f'Total training time: {total_training_time:.2f} seconds')
    return model_nn, total_training_time, avg_train_loss, avg_val_loss

In [ ]:
pool = nn.MaxPool2d(2, stride=2)
class CIFAR10_CNN(nn.Module):
    def __init__(self):
        super(CIFAR10_CNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3072, 768, kernel_size=3, padding=1),
            nn.ReLU(),
            pool,

            nn.Conv2d(768, 10, kernel_size=3, padding=1),
            nn.ReLU(),
            pool,

            nn.Conv2d(192, 10, kernel_size=3, padding=1),
            nn.ReLU(),
            pool
        )
        self.classifier = nn.Sequential(
            nn.Linear(128 * 4 * 4, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.LogSoftmax(dim=1)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

cifar_cnn_model1 = CIFAR10_CNN().to(device)
print(cifar_cnn_model1)

CIFAR10_CNN(
  (features): Sequential(
    (0): Conv2d(3072, 768, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(768, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Linear(in_features=2048, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=10, bias=True)
    (3): LogSoftmax(dim=1)
  )
)


In [ ]:
train_cifar_model1(n_epochs, optimizer, cifar_model1, transformed_cifar10_train, cifar10_val)

Epoch 300, Validation Loss: 2.9529, Validation Accuracy: 32.25%
Epoch 300, Training Loss: 0.0031
Total training time: 4370.77 seconds


(Sequential(
   (0): Linear(in_features=3072, out_features=512, bias=True)
   (1): ReLU()
   (2): Linear(in_features=512, out_features=10, bias=True)
   (3): LogSoftmax(dim=1)
 ),
 4370.7733907699585,
 0.0031042269858605493,
 2.9529001196478584)

**Part 1A: CNN Multilayer**

In [ ]:
learning_rate = 1e-2

optimizer = optim.SGD(cifar_model1.parameters(), lr=learning_rate)

loss_fn_cifar= nn.NLLLoss()

n_epochs = 300

def train_cifar_model2(n_epochs, optimizer, model_nn, train_dataset, val_dataset):
    start_time = time.time()

    # Create DataLoaders for batching
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=64, shuffle=False)

    for epoch in range(1, n_epochs + 1):
        model_nn.train() # Set model to training mode
        running_loss = 0.0
        for imgs, labels in train_loader:
            # Flatten images for the linear model
            imgs = imgs.view(imgs.shape[0], -1).to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            out = model_nn(imgs)
            loss = loss_fn_cifar(out, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        avg_train_loss = running_loss / len(train_loader)

        # Validation phase
        model_nn.eval() # Set model to evaluation mode
        val_running_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for imgs, labels in val_loader:
                imgs = imgs.view(imgs.shape[0], -1).to(device)
                labels = labels.to(device)
                out = model_nn(imgs)
                val_loss = loss_fn_cifar(out, labels)
                val_running_loss += val_loss.item()

                _, predicted = torch.max(out.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        avg_val_loss = val_running_loss / len(val_loader)
        val_accuracy = 100 * correct / total

    end_time = time.time()
    total_training_time = end_time - start_time
    if epoch % 20 == 0:
      print(f'Epoch {epoch}, Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%')
      print(f"Epoch {epoch}, Training Loss: {avg_train_loss:.4f}")
      print(f'Total training time: {total_training_time:.2f} seconds')
    return model_nn, total_training_time, avg_train_loss, avg_val_loss

In [ ]:
pool = nn.MaxPool2d(2, stride=2)
class CIFAR10_CNN(nn.Module):
    def __init__(self):
        super(CIFAR10_CNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3072, 1536, kernel_size=3, padding=1),
            nn.ReLU(),
            pool,

            nn.Conv2d(1536, 768, kernel_size=3, padding=1),
            nn.ReLU(),
            pool,

            nn.Conv2d(768, 384, kernel_size=3, padding=1),
            nn.ReLU(),
            pool,

            nn.Conv2d(384, 10, kernel_size=3, padding=1),
            nn.ReLU(),
            pool
        )
        self.classifier = nn.Sequential(
            nn.Linear(128 * 4 * 4, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.LogSoftmax(dim=1)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

cifar_cnn_model2 = CIFAR10_CNN().to(device)
print(cifar_cnn_model2)

CIFAR10_CNN(
  (features): Sequential(
    (0): Conv2d(3072, 1536, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(1536, 768, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(768, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Conv2d(384, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): ReLU()
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Linear(in_features=2048, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=10, bias=True)
    (3): LogSoftmax(dim=1)
  )
)


In [ ]:
train_cifar_model2(n_epochs, optimizer, cifar_model1, transformed_cifar10_train, cifar10_val)

Epoch 300, Validation Loss: 3.1682, Validation Accuracy: 31.99%
Epoch 300, Training Loss: 0.0012
Total training time: 4359.15 seconds


(Sequential(
   (0): Linear(in_features=3072, out_features=512, bias=True)
   (1): ReLU()
   (2): Linear(in_features=512, out_features=10, bias=True)
   (3): LogSoftmax(dim=1)
 ),
 4359.1521735191345,
 0.0012377238293658332,
 3.1681739296882774)

**Part 2A**

In [5]:
class BasicBlock(nn.Module):
    expansion = 1 # For ResNet, BasicBlock output channels are same as input channels

    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        # First convolutional layer
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)

        # Second convolutional layer
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        # Shortcut connection logic
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        identity = x # Store input for shortcut connection

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        out += self.shortcut(identity) # Add shortcut connection
        out = self.relu(out)

        return out

print("BasicBlock class defined successfully.")

BasicBlock class defined successfully.


In [6]:
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 64

        # Initial convolution layer
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)

        # ResNet layers
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)

        # Final classification layers
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)

        out = self.avgpool(out)
        out = torch.flatten(out, 1)
        out = self.fc(out)
        return out

print("ResNet class defined successfully.")

ResNet class defined successfully.


In [7]:
cifar_resnet10 = ResNet(BasicBlock, [1, 1, 1, 1], num_classes=10).to(device)
print(cifar_resnet10)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
  )
  (layer2): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(128, 12

In [8]:
from torchvision import datasets
import torchvision.transforms as transforms
import torch
import torch.nn as nn
import torch.optim as optim
import time

# Define data path
data_path = './data-unversioned/p1ch7/'

# Load datasets
cifar10_val = datasets.CIFAR10(data_path, train=False, download=True, transform=transforms.ToTensor())

transformed_cifar10_train = datasets.CIFAR10(data_path, train=True, download=True, transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4915,0.4823,0.4468), (0.2470,0.2435,0.2616))
]))

def train_resnet_model(n_epochs, optimizer, model, train_dataset, val_dataset, loss_fn):
    start_time = time.time()

    # Create DataLoaders
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=64, shuffle=False)

    for epoch in range(1, n_epochs + 1):
        model.train()
        running_loss = 0.0
        for imgs, labels in train_loader:
            # No flattening for ResNet/CNN
            imgs = imgs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            out = model(imgs)
            loss = loss_fn(out, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        avg_train_loss = running_loss / len(train_loader)

        # Validation phase
        model.eval()
        val_running_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for imgs, labels in val_loader:
                # No flattening
                imgs = imgs.to(device)
                labels = labels.to(device)
                out = model(imgs)
                val_loss = loss_fn(out, labels)
                val_running_loss += val_loss.item()

                _, predicted = torch.max(out.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        avg_val_loss = val_running_loss / len(val_loader)
        val_accuracy = 100 * correct / total

    end_time = time.time()
    total_training_time = end_time - start_time

    if epoch % 20 == 0:
      print(f'Epoch {epoch}, Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%')
      print(f"Epoch {epoch}, Training Loss: {avg_train_loss:.4f}")
      print(f'Total training time: {total_training_time:.2f} seconds')

    return model, total_training_time, avg_train_loss, avg_val_loss

# Configuration
learning_rate = 1e-2
optimizer_resnet = optim.SGD(cifar_resnet10.parameters(), lr=learning_rate)
# ResNet outputs logits, so we use CrossEntropyLoss (which combines LogSoftmax and NLLLoss)
loss_fn_resnet = nn.CrossEntropyLoss()
n_epochs = 300


In [ ]:
train_resnet_model(n_epochs, optimizer_resnet, cifar_resnet10, transformed_cifar10_train, cifar10_val, loss_fn_resnet)

**Part 2B**

In [9]:
# Experiment 1: Weight Decay with lambda = 0.001
print("Starting Experiment 1: Weight Decay (lambda=0.001)")

# Re-instantiate the standard ResNet model
model_wd = ResNet(BasicBlock, [1, 1, 1, 1], num_classes=10).to(device)

# Define optimizer with weight_decay
optimizer_wd = optim.SGD(model_wd.parameters(), lr=learning_rate, weight_decay=0.001)

# Train
# Note: Reusing train_resnet_model and loss_fn_resnet from previous cells
model_wd, time_wd, loss_wd, val_loss_wd = train_resnet_model(
    n_epochs, optimizer_wd, model_wd, transformed_cifar10_train, cifar10_val, loss_fn_resnet
)

Starting Experiment 1: Weight Decay (lambda=0.001)
Epoch 300, Validation Loss: 2.3330, Validation Accuracy: 32.09%
Epoch 300, Training Loss: 0.0062
Total training time: 6101.04 seconds


In [10]:
# Define BasicBlock with Dropout for Experiment 2
class BasicBlockDropout(nn.Module):
    expansion = 1

    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlockDropout, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)

        # Add Dropout with p=0.3
        self.dropout = nn.Dropout(p=0.3)

        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.dropout(out) # Apply Dropout

        out = self.conv2(out)
        out = self.bn2(out)

        out += self.shortcut(identity)
        out = self.relu(out)

        return out

print("BasicBlockDropout class defined.")

BasicBlockDropout class defined.


In [11]:
# Experiment 2: Dropout with p=0.3
print("Starting Experiment 2: Dropout (p=0.3)")

# Instantiate ResNet with BasicBlockDropout
model_dropout = ResNet(BasicBlockDropout, [1, 1, 1, 1], num_classes=10).to(device)

# Standard Optimizer (no weight decay)
optimizer_dropout = optim.SGD(model_dropout.parameters(), lr=learning_rate)

# Train
model_dropout, time_dropout, loss_dropout, val_loss_dropout = train_resnet_model(
    n_epochs, optimizer_dropout, model_dropout, transformed_cifar10_train, cifar10_val, loss_fn_resnet
)

Starting Experiment 2: Dropout (p=0.3)
Epoch 300, Validation Loss: 5.4335, Validation Accuracy: 24.58%
Epoch 300, Training Loss: 0.0035
Total training time: 6096.40 seconds
